In [1]:
#import pyspark

In [1]:
from pyspark import SparkContext, SparkConf

In [2]:
sc = SparkContext('local', 'First App')

In [3]:
numList = [2, 1, 4, 3]
nums = sc.parallelize(numList)
squarNum = nums.map(lambda x : x*x)
output = squarNum.collect()

max_s = nums.max()
min_s = nums.min()
sum_s = nums.fold(0, (lambda x, y: x+y))

print('max : '+str(max_s))
print('min : '+str(min_s))
print('sum : '+str(sum_s))

print('###### output ######')
for num in output :
    print(num)
print('####################')

max : 4
min : 1
sum : 10
###### output ######
4
1
16
9
####################


In [4]:
textFile = sc.textFile('README.md')

firstLine = textFile.first()
numOfLines = textFile.count()

SparkLine = textFile.filter(lambda line : 'Spark' in line)
numOfSparkLine = SparkLine.count()

print('firstLine : '+firstLine)
print('numOfLines : '+str(numOfLines))
print('numOfSparkLine : '+str(numOfSparkLine))

firstLine : This directory includes a few sample datasets to get you started.
numOfLines : 19
numOfSparkLine : 0


In [5]:
textFile = sc.textFile('README.md')
textFile_w = textFile.flatMap(lambda x : x.split(' '))
result = textFile_w.countByValue()

for key, value in result.items():
    print(key, value)

This 1
directory 1
includes 1
a 3
few 1
sample 2
datasets 1
to 1
get 1
you 1
started. 1
 51
* 3
`california_housing_data*.csv` 1
is 4
California 1
housing 1
data 1
from 1
the 3
1990 1
US 1
Census; 1
more 1
information 1
available 1
at: 2
https://developers.google.com/machine-learning/crash-course/california-housing-data-description 1
`mnist_*.csv` 1
small 1
of 2
[MNIST 1
database](https://en.wikipedia.org/wiki/MNIST_database), 1
which 1
described 2
http://yann.lecun.com/exdb/mnist/ 1
`anscombe.json` 1
contains 1
copy 2
[Anscombe's 1
quartet](https://en.wikipedia.org/wiki/Anscombe%27s_quartet); 1
it 1
was 2
originally 1
in 2
Anscombe, 1
F. 1
J. 1
(1973). 1
'Graphs 1
Statistical 1
Analysis'. 1
American 1
Statistician. 1
27 1
(1): 1
17-21. 1
JSTOR 1
2682899. 1
and 1
our 1
prepared 1
by 1
[vega_datasets 1
library](https://github.com/altair-viz/vega_datasets/blob/4f67bdaad10f45e3549984e17e1b3088c731503d/vega_datasets/_data/anscombe.json). 1


In [13]:
!wget -q http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\Gow/etc/wgetrc


In [14]:
!gzip -d kddcup.data_10_percent.gz

In [15]:
data_file = 'kddcup.data_10_percent'

raw_data = sc.textFile(data_file)

csv_data = raw_data.map(lambda l: l.split(','))

sampleLine = csv_data.first()
print(sampleLine)

['0', 'tcp', 'http', 'SF', '181', '5450', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '8', '8', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '9', '9', '1.00', '0.00', '0.11', '0.00', '0.00', '0.00', '0.00', '0.00', 'normal.']


In [16]:
# RDD to DataFrame
from pyspark.sql import Row
from pyspark.sql import SQLContext

row_data = csv_data.map(lambda p: Row(
    duration = int(p[0]),
    protocol_type = p[1],
    service = p[2],
    flag = p[3],
    src_bytes = int(p[4]),
    dst_bytes = int(p[5]),
    label = (p[41])
    )
)

sqlContext = SQLContext(sc)

df = sqlContext.createDataFrame(row_data)
df.printSchema()
df.show()
df.describe().show()

root
 |-- duration: long (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: long (nullable = true)
 |-- dst_bytes: long (nullable = true)
 |-- label: string (nullable = true)

+--------+-------------+-------+----+---------+---------+-------+
|duration|protocol_type|service|flag|src_bytes|dst_bytes|  label|
+--------+-------------+-------+----+---------+---------+-------+
|       0|          tcp|   http|  SF|      181|     5450|normal.|
|       0|          tcp|   http|  SF|      239|      486|normal.|
|       0|          tcp|   http|  SF|      235|     1337|normal.|
|       0|          tcp|   http|  SF|      219|     1337|normal.|
|       0|          tcp|   http|  SF|      217|     2032|normal.|
|       0|          tcp|   http|  SF|      217|     2032|normal.|
|       0|          tcp|   http|  SF|      212|     1940|normal.|
|       0|          tcp|   http|  SF|      159|     4087|norm

In [18]:
# Select duration & duration incremented by 1
df.select(df['duration'], df['duration'] + 1).show()
df.select(df['duration'], df['duration'] + 1).printSchema()

# Select duration greater than 0
df.filter(df['duration'] > 0).show()

# Count people by protocol_type
df.groupBy(df['protocol_type']).count().show()

+--------+--------------+
|duration|(duration + 1)|
+--------+--------------+
|       0|             1|
|       0|             1|
|       0|             1|
|       0|             1|
|       0|             1|
|       0|             1|
|       0|             1|
|       0|             1|
|       0|             1|
|       0|             1|
|       0|             1|
|       0|             1|
|       0|             1|
|       0|             1|
|       0|             1|
|       0|             1|
|       0|             1|
|       0|             1|
|       0|             1|
|       0|             1|
+--------+--------------+
only showing top 20 rows

root
 |-- duration: long (nullable = true)
 |-- (duration + 1): long (nullable = true)

+--------+-------------+--------+----+---------+---------+----------------+
|duration|protocol_type| service|flag|src_bytes|dst_bytes|           label|
+--------+-------------+--------+----+---------+---------+----------------+
|       1|          tcp|    smtp| 

In [22]:
df.registerTempTable('interactions')

selectSQL = sqlContext.sql('''
    SELECT duration, duration+1
    FROM interactions
''')
selectSQL.show()

whereSQL = sqlContext.sql('''
    SELECT *
    FROM interactions
    WHERE duration > 0
''')
whereSQL.show()

groupbySQL = sqlContext.sql('''
    SELECT protocol_type, count(protocol_type)
    FROM interactions
    GROUP BY protocol_type
''')
groupbySQL.show()

+--------+------------------------------+
|duration|(duration + CAST(1 AS BIGINT))|
+--------+------------------------------+
|       0|                             1|
|       0|                             1|
|       0|                             1|
|       0|                             1|
|       0|                             1|
|       0|                             1|
|       0|                             1|
|       0|                             1|
|       0|                             1|
|       0|                             1|
|       0|                             1|
|       0|                             1|
|       0|                             1|
|       0|                             1|
|       0|                             1|
|       0|                             1|
|       0|                             1|
|       0|                             1|
|       0|                             1|
|       0|                             1|
+--------+------------------------

In [23]:
# comparison
from time import time

t0 = time()
groupbySQL = sqlContext.sql('''
    SELECT protocol_type, count(protocol_type)
    FROM interactions
    GROUP BY protocol_type
''').show()
tt = time()- t0

print('Query performed in {} seconds'.format(round(tt,3)))

t0 = time()
df.select('protocol_type')\
    .groupBy('protocol_type')\
    .count()\
    .show()
tt = time()- t0

print('Query performed in {} seconds'.format(round(tt,3)))

+-------------+--------------------+
|protocol_type|count(protocol_type)|
+-------------+--------------------+
|          tcp|              190065|
|          udp|               20354|
|         icmp|              283602|
+-------------+--------------------+

Query performed in 5.901 seconds
+-------------+------+
|protocol_type| count|
+-------------+------+
|          tcp|190065|
|          udp| 20354|
|         icmp|283602|
+-------------+------+

Query performed in 5.712 seconds
